# Findings 
Here you find a summary of the initial findings

## Some stats

In [4]:
import pandas as pd
df = pd.read_csv('../data/archaeology_metadata.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158254 entries, 0 to 158253
Data columns (total 13 columns):
 #   Column                                                Non-Null Count   Dtype  
---  ------                                                --------------   -----  
 0   dsPersistentId                                        158254 non-null  object 
 1   publicationStatus                                     158254 non-null  object 
 2   title                                                 158254 non-null  object 
 3   dsDescriptionValue                                    158254 non-null  object 
 4   dansSpatialPointX                                     56479 non-null   object 
 5   dansSpatialPointY                                     56479 non-null   object 
 6   dansSpatialPointScheme                                56304 non-null   object 
 7   dansSpatialBoxNorth                                   4432 non-null    object 
 8   dansSpatialBoxEast                          

In total, the archaeology data station contains 158254 published datasets. 
About 60000 have geospatial information in the metadata, either in the form of points (x, y), or a bounding box. 

--> About* 1/3 of the datasets have spatial metadata available. 